In [7]:
import requests
import bs4
from bs4 import BeautifulSoup
import pandas as pd
from io import StringIO
import numpy as np
from requests.exceptions import ConnectionError
from requests.exceptions import ReadTimeout
import json
import urllib
import sqlite3
import random, time

In [ ]:
# 爬蟲前先進入以下網頁後 用F12 檢查取得COOKIE XCSRFTOKEN 還有 request api
# https://rent.591.com.tw/?kind=0&region=1
# request api
# https://rent.591.com.tw/home/search/rsList?is_new_list=1&type=1&kind=0&searchtype=1&region=1

In [ ]:
HEADERS = {
    'Accept': "application/json, text/javascript, */*; q=0.01",
    'Accept-Encoding': "gzip, deflate, br",
    'Accept-Language': "zh-TW,zh;q=0.9,en-US;q=0.8,en;q=0.7",
    'Connection': "keep-alive",
    # 要改COOKIE
    'Cookie': "is_new_index=1; is_new_index_redirect=1; T591_TOKEN=iul8nsc4j4tjh3ip5mhprdm4b5; tw591__privacy_agree=0; user_index_role=2; __auc=4a7899fa1750ddb5f883cbfded6; user_browse_recent=a%3A5%3A%7Bi%3A0%3Ba%3A2%3A%7Bs%3A4%3A%22type%22%3Bi%3A8%3Bs%3A7%3A%22post_id%22%3Bi%3A122584%3B%7Di%3A1%3Ba%3A2%3A%7Bs%3A4%3A%22type%22%3Bi%3A8%3Bs%3A7%3A%22post_id%22%3Bi%3A123836%3B%7Di%3A2%3Ba%3A2%3A%7Bs%3A4%3A%22type%22%3Bi%3A8%3Bs%3A7%3A%22post_id%22%3Bi%3A124564%3B%7Di%3A3%3Ba%3A2%3A%7Bs%3A4%3A%22type%22%3Bi%3A8%3Bs%3A7%3A%22post_id%22%3Bi%3A117694%3B%7Di%3A4%3Ba%3A2%3A%7Bs%3A4%3A%22type%22%3Bi%3A8%3Bs%3A7%3A%22post_id%22%3Bi%3A123821%3B%7D%7D; _ga_ZDKGFY9EDM=GS1.1.1611478834.1.1.1611478976.0; _ga=GA1.3.1262902872.1602255089; _ga=GA1.4.1262902872.1602255089; new_rent_list_kind_test=0; urlJumpIpByTxt=%E5%8F%B0%E5%8C%97%E5%B8%82; urlJumpIp=1; webp=1; PHPSESSID=9egvli8bgs975k3al6d957i3n1; _gid=GA1.3.1650815301.1617010887; _gat=1; _dc_gtm_UA-97423186-1=1; _gat_UA-97423186-1=1; last_search_type=1; _gid=GA1.4.1650815301.1617010887; XSRF-TOKEN=eyJpdiI6IkpEMEJtSXNYNldWallWSWhKTFwvOTlnPT0iLCJ2YWx1ZSI6Ik9XQkVlYUNXMGVWTnNXSlkrNzAyaitoNmsrcjRDY0FDdmNtenNoRjVkTmE2UmNTRlwvSHZhQUZNeWVzRkJCWUgweTl2emljUk9yNlJ3UHZtMnJ5VHlLdz09IiwibWFjIjoiOWEzZDQ4MzczOWFkYjBlNWYzNzZmZGMxMDA4YzU3MWRiNDgwZWQ0YWY2OGE0YTYyMTc2YjhlYmFmYzI1MTM0NyJ9; 591_new_session=eyJpdiI6Imw5T0tOSlJcL3c3Zk5BNTZScElBVXRnPT0iLCJ2YWx1ZSI6IjJNNERnbm02YWFoS2ZpXC9PbVpRcGFUMXExTFRBd3JJQUdaSzJFNjU5aUFPaWFFdXcwWEZmYnZEbVo5Ynd1ZVRoTVRlSGFrUElvR09FeWlNZU4yaFI3dz09IiwibWFjIjoiY2IxNjM4NTRjMzRlZTA1ZGY1NTgxMGY5NTAxYjFmNTNkYWVjOTA0NzI1NjNjNTVkNzZlODU5YTk5YWExYWZiNSJ9",
    'Referer': "https://rent.591.com.tw/?kind=0&region=1",
    'sec-ch-ua': "'Google Chrome';v='89', 'Chromium';v='89', ';Not A Brand';v='99'",
    'sec-ch-ua-mobile': "?0",
    'Sec-Fetch-Dest': "empty",
    'Sec-Fetch-Mode': "cors",
    'Sec-Fetch-Site': "same-origin",
    'User-Agent': "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.82 Safari/537.36",
    # 還有XCSRFTOKEN
    'X-CSRF-TOKEN': "YKsUTcDolgAVBoS4Lq0GAm9Jhu33lYQ8byKdnsnl",
    'X-Requested-With': "XMLHttpRequest",
}

In [ ]:
try:
    # 新的request api 放在以下
    url_api = 'https://rent.591.com.tw/home/search/rsList?is_new_list=1&type=1&kind=0&searchtype=1&region=3&firstRow=30'
    response = requests.get(url_api, headers=HEADERS)
    # print(response.headers)
    # 取得所有筆數
    parsed = json.loads(res.content)    
    total_records = parsed['records']
    records = int(total_records.replace(',', ''))
    alltimes = math.ceil(records/30) + 1
    print("成功取得總數量， 開始每三十筆的爬蟲")
    for i in range(30, alltimes*30, 30):
        url_real = url_api + '&firstRow=' + str(i)
        response = requests.get(url_real, headers=HEADERS)
        stop_sec = random.randint(1, 90)
        print(f'{i} / {records} 已Post，等待{stop_sec}秒')
        time.sleep(stop_sec)
        parsed = json.loads(res.content)
        
        # 每抓一個就先存檔
        with open('newdata.txt', 'w') as outfile:
            json.dump(parsed, outfile)
        outfile.close()
        # 打開檔案
        with open('newdata.txt') as json_file:
            data = json.load(json_file)
        json_file.close()
        
        detail_id_list = []
        detail_url_list = []
        nick_name_list = []
        for i in range(len(data['data']['data'])):
            # id for detail url
            detail_id = str(data['data']['data'][i]['id'])
            detail_url = 'https://rent.591.com.tw/rent-detail-' + detail_id + '.html'
            detail_id_list.append(detail_id)
            detail_url_list.append(detail_url)
            #print(data['data']['data'][i]['nick_name']) #出租者 與 出租者身分    
            nick_name = data['data']['data'][i]['nick_name']
            nick_name_list.append(nick_name)
            
        position_list = []
        firstname_list = []
        gender_list = []    
        for j in range(len(nick_name_list)):
            tmp_p = nick_name_list[j][0:3]
            position = tmp_p.strip()
            position_list.append(position)

            tmp_ng = nick_name_list[j][3:]
            ng = tmp_ng.strip()
            firstname = ng[0]
            tmp_gender = ng[1:]
            if tmp_gender == "先生":
                truely_gender = "1"
            elif tmp_gender == "小姐":
                truely_gender = "0"
            else:
                truely_gender = "2"

            firstname_list.append(firstname)
            gender_list.append(truely_gender) 
            
        # 存入DB  A TABLE
        db_root = "C:/Users/Morris/Desktop/RentalInfo.db"
        # c.execute('create table table_name (title, price)')
        conn = sqlite3.connect(db_root)
        c = conn.cursor()
        for k in range(len(detail_id_list)):
            id_name = detail_id_list[k]
            url_d = detail_url_list[k]
            nick_name = nick_name_list[k]
            position = position_list[k]
            firstname = firstname_list[k]
            gender = gender_list[k]
            insert_data = [id_name, nick_name, position, firstname, gender, url_d]
            # 寫入sqlite
            # 設定為 PK值 重複就不會插入資料
            try:
                c.execute('insert into DetailUrlFile values (?, ?, ?, ?, ?, ?)', insert_data)
                conn.commit()
            except:
                print("id Title duplicated")
                continue
            print("COMPLETE")    
        c.close()
except:
    print("Header 過期")

# 以下是分步解析

In [ ]:
with open('newdata.txt') as json_file:
    data = json.load(json_file)
json_file.close()
# print(json.dumps(data, indent=4, ensure_ascii=False, sort_keys=True)) #All data of 30
total_records = data['records']
# print(data['records'])  #該市 總件數
# print(len(data['data']['data'])) #我要的每30筆

# data['data']['data'] #我要的資料
detail_id_list = []
detail_url_list = []
nick_name_list = []
for i in range(len(data['data']['data'])):
    # id for detail url
    detail_id = str(data['data']['data'][i]['id'])
    detail_url = 'https://rent.591.com.tw/rent-detail-' + detail_id + '.html'
    detail_id_list.append(detail_id)
    detail_url_list.append(detail_url)
    #print(data['data']['data'][i]['nick_name']) #出租者 與 出租者身分    
    nick_name = data['data']['data'][i]['nick_name']
    nick_name_list.append(nick_name)


position_list = []
firstname_list = []
gender_list = []    
for j in range(len(nick_name_list)):
    tmp_p = nick_name_list[j][0:3]
    position = tmp_p.strip()
    position_list.append(position)
    
    tmp_ng = nick_name_list[j][3:]
    ng = tmp_ng.strip()
    firstname = ng[0]
    tmp_gender = ng[1:]
    if tmp_gender == "先生":
        truely_gender = "1"
    elif tmp_gender == "小姐":
        truely_gender = "0"
    else:
        truely_gender = "2"
    
    firstname_list.append(firstname)
    gender_list.append(truely_gender)

In [ ]:
# 存入DB  A TABLE
db_root = "C:/Users/Morris/Desktop/RentalInfo.db"
# c.execute('create table table_name (title, price)')
conn = sqlite3.connect(db_root)
c = conn.cursor()
for i in range(len(detail_id_list)):
    id_name = detail_id_list[i]
    url_d = detail_url_list[i]
    nick_name = nick_name_list[i]
    position = position_list[i]
    firstname = firstname_list[i]
    gender = gender_list[i]
    insert_data = [id_name, nick_name, position, firstname, gender, url_d]
    # 寫入sqlite
    # 設定為 PK值 重複就不會插入資料
    try:
        c.execute('insert into DetailUrlFile values (?, ?, ?, ?, ?, ?)', insert_data)
        conn.commit()
    except:
        print("id Title duplicated")
        continue
    print("COMPLETE")    
c.close()    

# B步驟

In [8]:
# B STEP 就不需要header了 不會記錄COOKIE
# From db to get detailURL

url_path_list = []
id_list = []
gender_demand_list = []
case_status_list = []
case_now_list = []
tel_list = []
status_list = []

db_root = "C:/Users/Morris/Desktop/RentalInfo.db"
conn = sqlite3.connect(db_root)
c = conn.cursor()
c.execute('select url, id from DetailUrlFile')
rows = c.fetchall()
c.close()
count = 0
for i in rows:
    count += 1
    urlpath = i[0]
    id_n = i[1]
    
    url_path_list.append(urlpath)
    id_list.append(id_n)
    
    res_detail = requests.get(urlpath)
    stop_sec = random.randint(1, 90)
    print(f'{count} / {len(rows)} 已request，等待{stop_sec}秒')
    time.sleep(stop_sec)
    
    res_html = res_detail.content
    soup = BeautifulSoup(res_html, 'html.parser')
    # print(soup.prettify)
    
    try:
        # 取得性別要求
        sexual = soup.findAll('ul', {'class': 'clearfix labelList labelList-1'})
        gender_title = soup.findAll('div', {'class': 'one'})
        if gender_title[7].getText() == "性別要求":
            # 獨立套房才有性別要求
            for s in sexual:
                x = s.findAll('em')
                # print(x[7].getText()) #性別要求套房才有
                gender_demand = x[7].getText().strip() #男 女 男女生皆可
                gender_demand_list.append(gender_demand)
        else:
            gender_demand = "男女生皆可"
            gender_demand_list.append(gender_demand)


        # 取得案件資訊    
        info = soup.findAll('div', class_='detailInfo clearfix')
        for j in info:
            y = j.findAll('li')
            if y[2].getText()[0:2] == "型態":
                # 有性別要求才會是2 3
                case_status = y[2].getText()[4:].strip() #型態
                case_now = y[3].getText()[4:].strip() #現況
                case_status_list.append(case_status)
                case_now_list.append(case_now)
            else:
                # 沒有性別要求就是3 4
                case_status = y[3].getText()[4:].strip() #型態
                case_now = y[4].getText()[4:].strip() #現況
                case_status_list.append(case_status)
                case_now_list.append(case_now)

        # 取得電話號碼    
        user = soup.findAll('div', class_='userInfo')
        for k in user:
            number = k.find('span', class_='dialPhoneNum').attrs
            tel = number['data-value'] #電話號碼
            tel_list.append(tel)
        status_list.append("上架中")
    except Exception as e:
        print(f"Error msg: {e} | {id_n} 此案已下架")
        gender_demand_list.append("-")
        case_status_list.append("-")
        case_now_list.append("-")
        tel_list.append("-")
        status_list.append("已下架")
        pass

1 / 30 已request，等待2秒
Error msg: list index out of range | 10559001 此案已下架
2 / 30 已request，等待31秒
3 / 30 已request，等待5秒
4 / 30 已request，等待13秒
5 / 30 已request，等待30秒
6 / 30 已request，等待11秒
7 / 30 已request，等待73秒
8 / 30 已request，等待15秒
9 / 30 已request，等待16秒
10 / 30 已request，等待27秒
11 / 30 已request，等待1秒
12 / 30 已request，等待55秒
13 / 30 已request，等待66秒
Error msg: list index out of range | 10540452 此案已下架
14 / 30 已request，等待62秒
15 / 30 已request，等待65秒
16 / 30 已request，等待62秒
17 / 30 已request，等待18秒
Error msg: list index out of range | 10635993 此案已下架
18 / 30 已request，等待50秒
Error msg: list index out of range | 10675867 此案已下架
19 / 30 已request，等待56秒
20 / 30 已request，等待87秒
21 / 30 已request，等待86秒
Error msg: list index out of range | 10540461 此案已下架
22 / 30 已request，等待14秒
23 / 30 已request，等待80秒
24 / 30 已request，等待2秒
25 / 30 已request，等待84秒
26 / 30 已request，等待33秒
Error msg: list index out of range | 10546773 此案已下架
27 / 30 已request，等待76秒
28 / 30 已request，等待36秒
29 / 30 已request，等待51秒
30 / 30 已request，等待7秒


In [9]:
db_root = "C:/Users/Morris/Desktop/RentalInfo.db"
# c.execute('create table table_name (title, price)')
conn = sqlite3.connect(db_root)
c = conn.cursor()

for i in range(len(id_list)):
    id_name = id_list[i]
    url_name = url_path_list[i]
    gender_de = gender_demand_list[i]
    case_s = case_status_list[i]
    case_n = case_now_list[i]
    tel_b = tel_list[i]
    status = status_list[i]
    insert_data = [id_name, gender_de, case_s, case_n, tel_b, status, url_name]
    # 寫入sqlite
    # 設定為 PK值 重複就不會插入資料
    try:
        c.execute('insert into UrlInsideContent values (?, ?, ?, ?, ?, ?, ?)', insert_data)
        conn.commit()
    except:
        print("id Title duplicated")
        continue
    print("COMPLETE")    
c.close()  

COMPLETE
COMPLETE
COMPLETE
COMPLETE
COMPLETE
COMPLETE
COMPLETE
COMPLETE
COMPLETE
COMPLETE
COMPLETE
COMPLETE
COMPLETE
COMPLETE
COMPLETE
COMPLETE
COMPLETE
COMPLETE
COMPLETE
COMPLETE
COMPLETE
COMPLETE
COMPLETE
COMPLETE
COMPLETE
COMPLETE
COMPLETE
COMPLETE
COMPLETE
COMPLETE
